# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> Data Science 2: Advanced Topics in Data Science
## Homework 5 Parts 2 & 3: Vision Transformers & Kaggle Competition


**Harvard University**<br/>
**Spring 2025**<br/>
**Instructors**: Pavlos Protopapas, Natesh Pillai, and Chris Gumb


<hr style="height:2pt">

In [ ]:
# RUN THIS CELL
import requests
from IPython.core.display import HTML
styles = requests.get(
    "https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/"
    "content/styles/cs109.css"
).text
HTML(styles)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import time

SEED = 109

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# measure notebook runtime
time_start = time.time()

<div style = "background: lightsalmon; border: thin solid black; border-radius: 2px; padding: 5px">

### Instructions
- To submit your notebook, follow the instructions given in on the Canvas assignment page.
- Plots should be legible and interpretable *without having to refer to the code that generated them*. They should includelabels for the $x$- and $y$-axes as well as a descriptive title and/or legend when appropriate.
- When asked to interpret a visualization, do not simply describe it (e.g., "the curve has a steep slope up"), but instead explain what you believe the plot *means*.
- Autograding tests are mostly to help you debug. The tests are not exhaustive so simply passing all tests may not be sufficient for full credit.
- The use of *extremely* inefficient or error-prone code (e.g., copy-pasting nearly identical commands rather than looping) may result in only partial credit.
- We have tried to include all the libraries you may need to do the assignment in the imports cell provided below. Please get course staff approval before importing any additional 3rd party libraries.
- Enable scrolling output on cells with very long output.
- Feel free to add additional code or markdown cells as needed.
- Ensure your code runs top to bottom without error and passes all tests by restarting the kernel and running all cells (note that this can take a few minutes).
- **You should do a "Restart Kernel and Run All Cells" before submitting to ensure (1) your notebook actually runs and (2) all output is visible**
</div>

<a id="contents"></a>

# Notebook Contents

- [**PART 2 [40 pts]: Vison Transformer from Scratch**](#part2)
- [**PART 3 [10 pts]: Kaggle Competition**](#part3)

<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">
    
# **PART 2: Vision Transformer from Scratch (40 points)**

Vision Transformers (ViTs) (for more details, see the original paper [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929)) represent a groundbreaking approach in computer vision by adapting the Transformer architecture, originally designed for natural language processing, to image recognition tasks. The key idea is to split an image into a sequence of patches, treat these patches as tokens (much like words in a sentence), and process them with Transformer models. This paradigm shift has enabled ViTs to achieve state-of-the-art performance on various tasks.

In this part of the homework, we will implement a Vision Transformer (ViT) from scratch using TensorFlow. When fine-grained control over the operations in your model is needed, standard Keras layers might not be flexible enough. Creating custom layers, on the other hand, allows you to directly manage each operation within the layer. This not only enables experimentation with novel ideas but also allows you to tailor the model's behavior to its specific requirements. We will build our model using custom layers and train it on a classification task.


---

#### Creating a custom layer

TensorFlow's design specifies that all custom layers should inherit from `tf.keras.layers.Layer` to ensure they integrate seamlessly into its ecosystem (e.g., automatic differentiation, weight management). Inheritance is a fundamental concept in object-oriented programming (OOP) that allows a class (the child or subclass) to inherit attributes and methods from another class (the parent or superclass).


There are **two essential methods** you must implement when creating a custom layer:

1. **Constructor (`__init__`)**  
   - **Purpose:** Sets up the layer.
   - **What to Do:**  
     - Inherit from `tf.keras.layers.Layer` by calling `super().__init__()` or `super(class_name, self).__init__()`.
     - Initialize any trainable parameters or sub-layers.

2. **`call`**  
   - **Purpose:** Defines the forward pass (i.e., how inputs are transformed into outputs).
   - **What to Do:**  
     - Implement the layer’s logic. Whatever computation you need (for example, an element-wise operation) should be written in this method.
     - This method is automatically invoked during model training and inference.


**Example**
```python
import tensorflow as tf

class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, hidden_dim):
        super(ResidualBlock, self).__init__()
        self.dense1 = tf.keras.layers.Dense(hidden_dim, activation='relu')
        self.dense2 = tf.keras.layers.Dense(hidden_dim, activation='relu')
        self.dense3 = tf.keras.layers.Dense(hidden_dim)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = x + inputs
        return x
```

**What Happens in __init__:**
- We call `super(ResidualBlock, self).__init__()` to inherit from `tf.keras.layers.Layer`.
- We initialize three Dense layers `dense1`, `dense2` and `dense3`, using ReLU activation in the first two, and linear in the last one.

**What Happens in call:**
- The `call()` method passes the input through the three Dense layers sequentially
- a skip connection adds the original input to the output of the seuqnece of Dense layers.

</div>

## The Quick, Draw! Dataset 

For this problem, we will be working with a modified version of the [The Quick, Draw! Dataset](https://github.com/googlecreativelab/quickdraw-dataset), which is a large database of 50 million drawings across 345 categories, collected from players of the game [Quick, Draw!](https://quickdraw.withgoogle.com/).

For this homework, we will simplify the problem by using only 306,000 images across 24 different classes for training (this includes the data you will use for validation) and a test set of 54,000 images.


<a id="part2data"></a>

## Downloading the Data Files


**The required `train.csv` file is available on [the "Data" tab of the CS1090B HW5 Kaggle Competition website](https://www.kaggle.com/t/e067286be2c54c078434793659326610). (DO NOT USE DATA FROM ANY OTHER SOURCE!)**


- `train.csv` is our training dataset and the `label` column contains our response class. The 784 other columns correspond to the pixel values of the 28x28 dimension image.  `train.csv` has 306,000 samples. This is the only file needed for this part of the homework.

## The Quick, Draw! Dataset 

[Return to contents](#contents)


![](img/quickdrawpreview.jpg)



For this problem, we will be working with a modified version of the [The Quick, Draw! Dataset](https://github.com/googlecreativelab/quickdraw-dataset), which is a large database of 50 million drawings across 345 categories, collected from players of the game [Quick, Draw!](https://quickdraw.withgoogle.com/).

For this homework, we will simplify the problem by using only 306,000 images across 24 different classes for training (this includes the data you will use for validation) and a test set of 54,000 images.


<a id="part2data"></a>

## Downloading the Data Files

[Return to contents](#contents)


**The required `train.csv` and `test.csv` data files are available on [the "Data" tab of the CS1090B HW5 Kaggle Competition website](https://www.kaggle.com/t/e067286be2c54c078434793659326610). (DO NOT USE DATA FROM ANY OTHER SOURCE!)** Also please check the overview section for important notes [https://www.kaggle.com/competitions/cs-1090-b-hw-5-contest-2025/overview](https://www.kaggle.com/competitions/cs-1090-b-hw-5-contest-2025/overview)

Here's a brief description of the data files:

- `train.csv` is our training dataset and the `label` column contains our response class. The 784 other columns correspond to the pixel values of the 28x28 dimension image.  `train.csv` has 306,000 samples.

- `test.csv` file mirrors the structure of the `train.csv` file **without** the class label column. We have added another column `id`. `id` is a unique identifier for each sample in the test set. This ID is crucial for mapping your model's predictions back to the corresponding samples when submitting your results to Kaggle. `test.csv` has 54,000 samples. 


In [ ]:
def load_data_from_csv(data_file):
    """Load and preprocess data from CSV file"""
    df = pd.read_csv(data_file)

    classes = sorted(df['label'].unique().tolist())
    
    label_to_id = {label: idx for idx, label in enumerate(classes)}
    
    X = df.drop('label', axis=1).values.astype('uint8')
    y = df['label'].map(label_to_id).values 
    
    # Normalize pixel values to [0, 1]
    X = X.astype('float32') / 255.0
    X = X.reshape(-1, 28, 28, 1)
    
    return df, X, y, classes, label_to_id  

# Load and preprocess the data
data_file = "train.csv"
df, X, y, classes, label_to_id = load_data_from_csv(data_file) 

In [ ]:
def plot_image_grid(df, classes, rows=5, cols=5):
    """
    Plot a grid of images for each class from the CSV data
    
    Args:
        df: DataFrame containing the image data and labels
        classes: List of class names
        rows: Number of rows in the grid (default 5)
        cols: Number of columns in the grid (default 5)
    """
    # Calculate figure size with reduced height per class
    fig_height = len(classes) * 1.5  
    fig = plt.figure(figsize=(15, fig_height))
    
    # Adjust the spacing between subplots
    plt.subplots_adjust(hspace=0.4, wspace=0.1)
    
    for idx, class_name in enumerate(classes):
        class_samples = df[df['label'] == class_name].iloc[:rows*cols]
        
        for i in range(rows*cols):
            if i < len(class_samples):
                # Get image data, convert to float32 and normalize to [0,1]
                img_data = class_samples.iloc[i].drop(['label']).values.astype('float32') / 255.0
                img_data = img_data.reshape(28, 28)
                
                # Create subplot
                ax = plt.subplot(len(classes), rows*cols, idx*(rows*cols) + i + 1)
                plt.imshow(img_data, cmap='gray')
                plt.axis('off')
                
                # Add class label to the first image of each row
                if i == 0:
                    ax.text(-0.5, 0.5, class_name, rotation=0, 
                           ha='right', va='center', transform=ax.transAxes)
    
    plt.tight_layout()
    plt.show()

plot_image_grid(df, df.label.unique())

<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">
    
#### Q2.1 Single-Head Self Attention

We will build each component of ViT separetely and combine everything in the `VisionTransformer` class. But let's start by implementing self-attention using a custom layer. Define a `SingleHeadAttention` class with the following:


**In the `__init__` method** fill in the blanks to initialize:
   - `W_query`: A dense layer mapping inputs to tensors with dimension `hidden_dim`
   - `W_key`: A dense layer mapping inputs to tensors with dimension `hidden_dim`
   - `W_value`: A dense layer mapping inputs to tensors with dimension `hidden_dim`
   - `hidden_dim`: The dimension of the projected tensors.

**In the `call` method you have to** compute the attention output. As you can recall form the "Attention Is All You Need" paper. Given an input tensor with shape `(batch_size, seq_length, input_dim)`, a Single-Head Self Attention layer performs the following operations:
   1. **Linear Projections**: Project the input into queries, keys, and values using learned weight matrices:
      $$Q = \text{inputs} \cdot W_{\text{query}}$$
      $$K = \text{inputs} \cdot W_{\text{key}}$$
      $$V = \text{inputs} \cdot W_{\text{value}}$$
      
      Where each projection transforms from `input_dim` to `hidden_dim`.

   2. **Scaled Dot-Product Attention**: Compute attention scores:
      $$\text{attention\_scores} = \frac{Q \cdot K^T}{\sqrt{\text{hidden\_dim}}}$$
      $$\text{attention\_weights} = \text{softmax}(\text{attention\_scores})$$
      $$\text{attention\_output} = \text{attention\_weights} \cdot V$$


**NOTE**: TensorFlow’s Dense layers perform “lazy” weight initialization – meaning that the layer’s weights aren’t created until the first time the layer is called (when the input shape is known). Therefore, you do not need to know `input_dim` when creating the layer, this will be defined by the input shape, when given to the model.

</div>

In [ ]:
class SingleHeadAttention(tf.keras.layers.Layer):
    def __init__(self, hidden_dim):
        """
        Single-head attention layer.
        
        Args:
            hidden_dim: Dimensionality for queries, keys, and values.
        """

        super(SingleHeadAttention, self).__init__()
        # your code

    def call(self, inputs):
        """
        Forward pass for single-head attention.
        
        Args:
            inputs: Tensor of shape (batch_size, seq_length, input_dim).
            
        Returns:
            attention_output: Attention output of shape (batch_size, seq_length, hidden_dim).
        """
        # your code here


#### Test 1: `SingleHeadAttention` implementation

In [ ]:
# Run the following test to help debug your implementation so far

batch_size = 2
seq_length = 8
input_dim = 64
hidden_dim = 32

# Create random input
inputs = tf.random.normal((batch_size, seq_length, input_dim))

attention = SingleHeadAttention(hidden_dim=hidden_dim)

# Forward pass: note that attention(inputs) is equivalent to attention.call(inputs)
attention_output = attention(inputs)

expected_output_shape = (batch_size, seq_length, hidden_dim)

assert attention_output.shape == expected_output_shape, \
    f"Output shape mismatch: got {attention_output.shape}, expected {expected_output_shape}"

print(f"SingleHeadAttention output shape: {attention_output.shape}")

<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">

#### Q2.2 Multi-Head Attention Layer

The `MultiHeadAttentionLayer` implements multi-head self-attention by processing several single-head attention operations in parallel and then combining their outputs. Given an input tensor with shape `(batch_size, seq_length, input_dim)`, follow these instructions:

**In the `__init__` method, you need to:**

- **Initialize the number of heads and single-head attention layers:**
  - Create a list of attention heads. Each head is an instance of your single-head attention layer, configured with a projection output dimensionality of `head_dim`.  
  - For each head $i$ (with $i = 1, \dots, \text{num\_heads}$), the head computes  
    $$
    \text{attn\_output}_i = \text{SingleHeadAttention}_i(\text{inputs})
    $$
    where each $\text{attn\_output}_i$ has shape `(batch_size, seq_length, head_dim)`.

- **Initialize the output projection layer:**
  - Define a Dense layer that projects the concatenated outputs back to the original input dimension (`input_dim`).  
  - Once you have concatenated the outputs from all heads, you obtain a tensor of shape `(batch_size, seq_length, head_dim x num_heads)` and the output projection layer maps this tensor to shape `(batch_size, seq_length, input_dim)`.

**In the `call` method, you need to:**

1. **Process each attention head:**
   - Iterate over the list of single-head attention layers and pass the input tensor through each one. This produces, for each head $i$, an output $\text{attn\_output}_i$ of shape `(batch_size, seq_length, head_dim)`.

2. **Concatenate the outputs:**
   - Concatenate all the $\text{attn\_output}_i$ tensors along the last (feature) axis:
     $$
     \text{concatenated} = \text{concat}\left(\left[\text{attn\_output}_1,\, \text{attn\_output}_2,\, \dots,\, \text{attn\_output}_{\text{num\_heads}}\right]\right)
     $$
     This results in a tensor of shape `(batch_size, seq_length, head_dim × num_heads)`.

3. **Apply the output projection:**
   - Feed the concatenated tensor into the output projection Dense layer:
     $$
     \text{multihead\_output} = \text{output\_projection}(\text{concatenated})
     $$
     This yields the final output tensor with shape `(batch_size, seq_length, input_dim)`.

This process enables the model to attend to different subspaces of the input simultaneously, thereby capturing a richer representation of the data.

</div>


In [ ]:
class MultiHeadAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, input_dim, head_dim, num_heads):
        """
        Multi-head attention layer.
        
        Args:
            input_dim: Dimensionality of the input and final output.
            head_dim: Dimensionality for each attention head.
            num_heads: Number of attention heads.
        """
        super(MultiHeadAttentionLayer, self).__init__()
        # your code here

    def call(self, inputs):
        """
        Forward pass for multi-head attention.
        
        Args:
            inputs: Tensor of shape (batch_size, seq_length, input_dim).
            
        Returns:
            multihead_output: Multi-head attention output of shape (batch_size, seq_length, input_dim).
        """
        # your code here

#### Test 2: `MultiHeadAttentionLayer` implementation

In [ ]:
# Run the following test to help debug your implementation so far

batch_size = 2
seq_length = 8
input_dim = 64
head_dim = 16
num_heads = 4

# Create random input tensor
inputs = tf.random.normal((batch_size, seq_length, input_dim))

multihead_attention = MultiHeadAttentionLayer(input_dim=input_dim, head_dim=head_dim, num_heads=num_heads)

attention_output = multihead_attention(inputs)

expected_output_shape = (batch_size, seq_length, input_dim)

assert attention_output.shape == expected_output_shape, (
    f"Output shape mismatch: got {attention_output.shape}, expected {expected_output_shape}"
)

print(f"Multi-head attention output shape: {attention_output.shape}")

<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">

#### Q2.3 Transformer Encoder Block

Given an input tensor `inputs` of shape `(batch_size, seq_length, embed_dim)` the `TransformerEncoderBlock` performs the following operations:

**In the `__init__` method, you need to:**

- **Initialize the Multi-Head Attention:**
  - Create a `MultiHeadAttentionLayer` that processes the inputs to produce `attn_output = MultiHeadAttentionLayer(inputs)`

- **Initialize the Feed-Forward Network:**
  - Build a Sequential block consisting of:
    - `LayerNormalization` with `epsilon = 1e-5`
    - A `Dense` layer with `GELU` activation mapping to `feedforward_dim` units
    - A `Dropout` layer with `ffn_dropout_rate`.
    - A `Dense` layer mapping back to `embed_dim`
    - A `Dropout` layer with `ffn_dropout_rate`.
  This block, when applied to an input \(x\), computes:
    $$
    \text{ffn\_output} = \text{FFN}(x)
    $$
    yielding a tensor with shape `(batch_size, seq_length, embed_dim)`.

**In the `call` method, you need to:**

1. **Compute the Multi-Head Attention:**
   - Apply layer norm to `inputs` with `epsilon = 1e-5`:
     $$
     \text{normalized\_inputs} = \text{LayerNormalization}(\text{inputs})
     $$
   - Apply the multi-head attention layer:
     $$
     \text{attn\_output} = \text{MultiHeadAttentionLayer}(\text{normalized\_inputs})
     $$
   - Apply `Dropout` with rate `attn_dropout_rate`:
     $$
     \text{attn\_output} = \text{Dropout}(\text{attn\_output})
     $$
   - Add a residual connection:
     $$
     x = \text{inputs} + \text{attn\_output}
     $$

2. **Process through the Feed-Forward Network:**
   - Compute the feed-forward output:
     $$
     \text{ffn\_output} = \text{FFN}(x)
     $$
   - Apply a second residual connection:
     $$
     \text{output} = x + \text{ffn\_output}
     $$

The final output tensor `output` of shape `(batch_size, seq_length, embed_dim)` incorporates both self-attention and feed-forward processing with residual connections.

</div>



In [ ]:
class TransformerEncoderBlock(tf.keras.layers.Layer):
    def __init__(self,
                 embed_dim,
                 attention_dim,
                 feedforward_dim,
                 num_heads,
                 ffn_dropout_rate=0.1,
                 attn_dropout_rate=0.1,
                 epsilon=1e-5):
        """
        A single Transformer encoder block with multi-head attention and feed-forward network.
        
        Args:
            embed_dim: Dimensionality of the input embeddings.
            attention_dim: Dimensionality used in each attention head.
            feedforward_dim: Dimensionality of the intermediate feed-forward layer.
            num_heads: Number of attention heads.
        """
        
        super(TransformerEncoderBlock, self).__init__()
        # your code here

    def call(self, inputs):
        """
        Forward pass for a single Transformer encoder block.
        
        Args:
            inputs: Tensor of shape (batch_size, seq_length, embed_dim).
        
        Returns:
            output: Output tensor of shape (batch_size, seq_length, embed_dim).
        """
        # your code here


#### Test 3: `TransformerEncoderBlock` implementation

In [ ]:
# Run the following test to help debug your implementation so far

batch_size = 2
seq_length = 8
embed_dim = 32
attention_dim = 16
feedforward_dim = 64
num_heads = 4

# Create a random input tensor
inputs = tf.random.normal((batch_size, seq_length, embed_dim))

encoder_block = TransformerEncoderBlock(
    embed_dim=embed_dim,
    attention_dim=attention_dim,
    feedforward_dim=feedforward_dim,
    num_heads=num_heads)

output = encoder_block(inputs)

expected_output_shape = (batch_size, seq_length, embed_dim)

assert output.shape == expected_output_shape, (
    f"Output shape mismatch: got {output.shape}, expected {expected_output_shape}"
)

print(f"TransformerEncoderBlock output shape: {output.shape}")

<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">

#### Q2.4 Transformer Encoder

Given an input tensor `inputs` of shape `(batch_size, seq_length, embed_dim)`, the `TransformerEncoder` applies a stack of encoder blocks sequentially. 

**In the `__init__` method, you need to:**

- **Initialize the Encoder Blocks:**
  - Create a list of `num_blocks` instances of `TransformerEncoderBlock`, each instantiated with parameters `embed_dim`, `attention_dim`, `feedforward_dim`, and `num_heads`.

**In the `call` method, you need to:**

1. **Initialize the Representation:**
   - Set
     $$
     x_0 = \text{inputs} \in \mathbb{R}^{\text{batch\_size} \times \text{seq\_length} \times \text{embed\_dim}}.
     $$

2. **Apply Each Encoder Block:**
   - For each encoder block $i$, with $i = 1, 2, \dots, \text{num\_blocks}$:
     $$
     x_i = \text{TransformerEncoderBlock}_i\big(x_{i-1}\big).
     $$
     
3. **Return the Final Output:**
   - The final output is 
     $$
     \text{output} = x_{\text{num\_blocks}} \in \mathbb{R}^{\text{batch\_size} \times \text{seq\_length} \times \text{embed\_dim}}.
     $$

This stacking operation iteratively refines the input representation through multiple encoder blocks.

</div>



In [ ]:
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, attention_dim, feedforward_dim, num_heads, num_blocks):
        """
        A stack of Transformer encoder blocks.
        
        Args:
            embed_dim: Dimensionality of the input embeddings.
            attention_dim: Dimensionality used in each attention head.
            feedforward_dim: Dimensionality of the intermediate feed-forward layer.
            num_heads: Number of attention heads.
            num_blocks: Number of encoder blocks.
        """
        
        super(TransformerEncoder, self).__init__()
        # BEGIN SOLUTION
        
        # END SOLUTION

    def call(self, inputs):
        """
        Forward pass for the Transformer encoder.
        
        Args:
            inputs: Tensor of shape (batch_size, seq_length, embed_dim).
        
        Returns:
            output: The final output of the encoder (batch_size, seq_length, embed_dim).
        """
        # your code here


#### Test 4: `TransformerEncoder` implementation

In [ ]:
# Run the following test to help debug your implementation so far

batch_size = 2
seq_length = 8
embed_dim = 32
attention_dim = 16
feedforward_dim = 64
num_heads = 4
num_blocks = 3

# Create a random input tensor of shape
inputs = tf.random.normal((batch_size, seq_length, embed_dim))

encoder = TransformerEncoder(embed_dim, attention_dim, feedforward_dim, num_heads, num_blocks)

output = encoder(inputs)

expected_output_shape = (batch_size, seq_length, embed_dim)

assert output.shape == expected_output_shape, (
    f"Output shape mismatch: got {output.shape}, expected {expected_output_shape}"
)

print(f"TransformerEncoder output shape: {output.shape}")

<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">

#### Q2.5 Patch Embedding

The `PatchEmbedding` layer converts an input image into a sequence of patch embeddings. Given an image tensor with shape `(batch_size, image_size, image_size, input_channels)`, the layer performs the following steps:

**In the `__init__` method, you need to:**

- **Store Relevant Attributes:**  
  - Save `image_size`, `input_channels`, `patch_size`, and `num_patches` as class attributes.
- **Compute Number of Patches:**  
  - Calculate `num_patches` as `(image_size // patch_size) ** 2`.
- **Initialize the Projection:**  
  - Create a Dense layer (`self.projection`) that will map the flattened patch vector to a vector of dimension `embed_dim`.

**In the `call` method, you need to:**

1. **Extract Patches:**  
   - Use `tf.image.extract_patches` to split the input image into non-overlapping patches. The patches are extracted with:
     - `sizes = [1, patch_size, patch_size, 1]`
     - `strides = [1, patch_size, patch_size, 1]`
     - `rates = [1, 1, 1, 1]`
     - `padding = "VALID"`
   - This produces a tensor of shape `(batch_size, image_size/patch_size, image_size/patch_size, patch_area * input_channels)`.

2. **Reshape the Patches:**  
   - Flatten the spatial dimensions to convert the patches tensor to shape `(batch_size, num_patches, patch_flat_dim)`, where:
     - `patch_flat_dim = patch_size * patch_size * input_channels`.
   - **Note**: You might find the function `tf.reshape()` useful in this case.

3. **Project the Patches:**  
   - Pass the flattened patches through the `Dense` projection layer to obtain the final patch embeddings.
   - The output has shape `(batch_size, num_patches, embed_dim)`.

This layer effectively transforms each patch (a flattened vector of pixel values) into a learned embedding, serving as the input to subsequent Transformer blocks.

</div>


In [ ]:
class PatchEmbedding(tf.keras.layers.Layer):
    def __init__(self, image_size, patch_size, input_channels, embed_dim):
        """
        Converts an image into a sequence of patch embeddings.
        
        Args:
            image_size: Size (height/width) of the input image (assumed square).
            patch_size: Size of each (square) patch.
            input_channels: Number of channels in the input image.
            embed_dim: Dimensionality of the patch embeddings.
        """
        
        super(PatchEmbedding, self).__init__()
        # your code here

    def call(self, images):
        """
        Forward pass for patch embedding.
        
        Args:
            images: Tensor of shape (batch_size, height, width, input_channels).
        
        Returns:
            A tensor of shape (batch_size, num_patches, embed_dim).
        """
        # your code here


#### Test 5: `PatchEmbedding` implementation

In [ ]:
# Run the following test to help debug your implementation so far 

batch_size = 2
image_size = 32
patch_size = 8
input_channels = 3
embed_dim = 64

num_patches = (image_size // patch_size) ** 2

# Create a random input tensor representing images
images = tf.random.normal((batch_size, image_size, image_size, input_channels))

patch_embedding = PatchEmbedding(
    image_size=image_size,
    patch_size=patch_size,
    input_channels=input_channels,
    embed_dim=embed_dim
)

output = patch_embedding(images)

expected_output_shape = (batch_size, num_patches, embed_dim)

assert output.shape == expected_output_shape, (
    f"Output shape mismatch: got {output.shape}, expected {expected_output_shape}"
)

print(f"PatchEmbedding output shape: {output.shape}")

<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">

#### Q2.6 Vision Transformer

The `VisionTransformer` model will put all these pieces together. To help a little bit, we filled the constructor for you. Given an input tensor with shape  `(batch_size, height, width, input_channels)` where `height = width = image_size`, the model operates as follows:

**In the `__init__` method:**

- **Determine Input Parameters:**
  - Extract `image_size` and `input_channels` from `input_size`.
- **Create the Patch Embedding Layer:**
  - The `PatchEmbedding` layer splits the image into non-overlapping patches and projects each patch to an embedding vector of dimension `embed_dim`.
  - The total number of patches is computed as `num_patches = (image_size // patch_size)^2`
- **Define Positional Embedding:**
  - Create a positional embedding using `tf.keras.layers.Embedding` with an input dimension equal to `num_patches` and an output dimension `embed_dim`.
- **Define a Learnable Class Token:**
  - Initialize a class token of shape `(1, 1, embed_dim)`, which is later broadcasted to every batch.
- **Instantiate the Transformer Encoder:**
  - Create a `TransformerEncoder` that contains `num_blocks` encoder blocks.  
    Each block applies multi-head attention and a feed-forward network (with their respective dropout, residual connections, and layer normalization) to process the input.
- **Define the Classification Head:**
  - Build a sequential model comprising a `LayerNormalization` (with `epsilon=1e-5`) followed by a Dense layer mapping to `num_classes`.

**In the `call` method, you need to:**

1. **Compute Patch Embeddings:**
   - Pass `images` through `self.patch_embedding` to obtain `patch_embeddings` of shape `(batch_size, num_patches, embed_dim)`
2. **Add Positional Embeddings:**
   - Generate a sequence of positions (of length `num_patches`) and map them through `self.position_embedding` to obtain positional embeddings:  
     `positional_embeddings` of shape `(1, num_patches, embed_dim)`
   - Add these positional embeddings to `patch_embeddings` elementwise.
3. **Prepend the Class Token:**
   - Broadcast the learnable class token to match the batch size, resulting in a tensor of shape `(batch_size, 1, embed_dim)`.
   - Concatenate this class token with the positional-enhanced patch embeddings along the sequence dimension, yielding:  
     `transformer_input` with shape `(batch_size, num_patches + 1, embed_dim)`
4. **Apply the Transformer Encoder:**
   - Pass `transformer_input` through `self.transformer_encoder` which outputs `encoder_output` with shape `(batch_size, num_patches + 1, embed_dim)`
5. **Classification:**
   - The classification head uses the embedding corresponding to the class token (index 0) to compute logits `logits`, with shape`(batch_size, num_classes)`

The model thus returns the `logits` that represent the class scores for each input image.

</div>



In [ ]:
class VisionTransformer(tf.keras.Model):
    # We'll give you the init ;)
    def __init__(
        self,
        num_classes,
        patch_size,
        num_heads,
        num_blocks,
        embed_dim,
        attention_dim,
        feedforward_dim,
        input_size=(28,28,1)
    ):
        """
        Vision Transformer model implementation.
        
        Args:
            num_classes: Number of output classes.
            patch_size: Size of each patch.
            num_heads: Number of attention heads.
            num_blocks: Number of Transformer encoder blocks.
            embed_dim: Dimensionality of the patch/position embeddings.
            attention_dim: Dimensionality used in each attention head.
            feedforward_dim: Dimensionality of the intermediate feed-forward layer.
            input_size: Dimensionality of the input
        """
        super(VisionTransformer, self).__init__()
        image_size = input_size[0]
        input_channels = input_size[-1]
        self.patch_embedding = PatchEmbedding(
            image_size, patch_size, input_channels, embed_dim
        )
        self.num_patches = (image_size // patch_size) ** 2
        self.embed_dim = embed_dim

        # Positional embedding for each patch
        self.position_embedding = tf.keras.layers.Embedding(
            input_dim=self.num_patches,
            output_dim=embed_dim
        )

        # Learnable class token used for classification
        self.cls = self.add_weight(
            "cls",
            shape=(1, 1, embed_dim),
            initializer=tf.random_normal_initializer()
        )

        self.transformer_encoder = TransformerEncoder(
            embed_dim=embed_dim,
            attention_dim=attention_dim,
            feedforward_dim=feedforward_dim,
            num_heads=num_heads,
            num_blocks=num_blocks
        )

        # Classification head: LayerNorm -> Dense
        self.classification_head = tf.keras.Sequential([
            tf.keras.layers.LayerNormalization(epsilon=1e-5),
            tf.keras.layers.Dense(num_classes)
        ])

    def call(self, images):
        """
        Forward pass through the Vision Transformer model.
        
        Args:
            images: Tensor of shape (batch_size, height, width, input_channels).
        
        Returns:
            logits: Logits of shape (batch_size, num_classes).
        """
        # your code here


<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">

#### 2.7 Model training and evaluation

##### Q2.7.1 Create data splits
   - Create train and validation splits from your dataset `(X, y)` using `train_test_split` with `test_size=0.1` and `random_state=109`.
   - Create the training and validation datasets using `tf.data.Dataset.from_tensor_slices()`, then batch them using a batch size of 512.

In [ ]:
# your code here


<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">


##### Q2.7.2 Model training and evaluation 
   Create a `VisionTransformer` with the following configuration:
   - `num_classes = 24`
   - `patch_size = 4`
   - `num_heads = 8`
   - `num_blocks = 4`
   - `embed_dim = 64`
   - `attention_dim = 64`
   - `feedforward_dim = 128`

After instantiating your model, call the `build` method with an input shape of `(None, 28, 28, 1)` and show a summary of it.

In [ ]:
# your code here


<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">

##### Q2.7.3 Model training

Train the model for 20 epochs using `Adam` with learning rate of 0.005, `SparseCategoricalCrossentropy` as loss function, and `accuracy` as metric.

In [ ]:
# your code here


<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">

#### Q2.7.4 Model Evaluation

Visualize the training progress by plotting both the training and validation loss and accuracy over the epochs.

In [ ]:
# your code here


<a id="part2"></a>
<div class="alert alert-success" style="color: #333; background-color: #e6ffe6; border-color: #c3e6cb; border-width: 1px; border-radius: 3px; padding: 30px;">

Then, evaluate the model on the training and validation sets and print the corresponding loss and accuracy values. You should get a validation accuracy >= 77 if you did everything right.

In [ ]:
# your code here


<a id="part3"></a>
# PART 3 [10 pts]: The Quick, Draw!  Kaggle competition

[Return to contents](#contents)

<a id="part3kaggle"></a>

## CS1090B Kaggle Competition

[Return to contents](#contents)

**ACCESS AND JOIN THE COMPETITION**:

**You need to create an account on Kaggle and [join the competition via this link](https://www.kaggle.com/t/e067286be2c54c078434793659326610). This is a limited participation competition. Please DO NOT share this link.**

**For more information on the rules** governing this CS1090B Kaggle competition, please see below and also review. The modeling restrictions DOS and DON'TS are outlined in Kaggle Overview page.

**IMPORTANT NOTES ABOUT SCORING**:

- `sample_submission.csv` is the format that kaggle will accept. The uploaded `.csv` must contain 2 columns. The first column must be named `id` and needs to contain the test observation index numbers for each prediction, the second must be named `label` and needs to contain your class predictions (i.e. 'cat', 'tiger' etc) for each corresponding test observation index location. 
- The **public leaderboard** on Kaggle displays your performance on 45% of the test set.


- After the competition is complete, the **private leaderboard** will show your performance on the remaining 55% of the test set.


**ADDITIONAL COMPETITION RULES:**

- Multiple Kaggle submissions are permitted (with a maximum of 5 submissions per team per-day), **just note that you will need to choose, on Kaggle, the ONE single submission to use for final scoring prior to the final HW5 submission deadline**, and **your submitted notebook MUST contain the matching code and model that generated your ONE chosen submission.**


- **To repeat this point, the version of your final HW5 notebook submitted on Canvas MUST contain the same code and exact same model used to generate your ONE chosen Kaggle submission.** (TFs may rerun your notebook code to ensure comparable final leaderboard results.)


- **Please do not manually label your submissions.** In other words, the labels should only be the outcome of your model.


- **No external data are allowed, you MUST USE ONLY the training and test data downloaded via the "Data" tab of [the CS1090B competition page linked above](#part2data).**

- **Do not** create multiple accounts on Kaggle.



<div class="alert alert-success" style="color: #333; background-color: #e6e6fa; border-color: #d8bfd8; border-width: 1px; border-radius: 3px; padding: 10px;">

<a id="q3.1"></a>

**3.1.1**  **Kaggle Competition**

Create a model and use it to compete on Kaggle.


**IMPORTANT: YOU MUST** ensure that the version of the code and model in your final submitted notebook is the **EXACT SAME** code and model used to generate your Kaggle submission. TFs may run your submitted model to ensure comparable results.

**Other Kaggle competition rules and scoring details [are listed here](#part2kaggle).
[https://www.kaggle.com/competitions/cs-1090-b-hw-5-contest-2025/overview](https://www.kaggle.com/competitions/cs-1090-b-hw-5-contest-2025/overview)**

</div>

In [2]:
# your code here



<div class="alert alert-success" style="color: #333; background-color: #e6e6fa; border-color: #d8bfd8; border-width: 1px; border-radius: 3px; padding: 10px;">


**3.1.2**  Plot your model's training accuracy and and any relevant metrics as a function of epochs.




</div>

In [3]:
# your code here



<div class="alert alert-success" style="color: #333; background-color: #e6e6fa; border-color: #d8bfd8; border-width: 1px; border-radius: 3px; padding: 10px;">


**3.1.3**  In a few sentences, describe the various approaches you have taken to improve the performance of your model as well as any observations you might have regarding your training and Kaggle results.


</div>

**APPROACH AND OBSERVATIONS:**

*your answer here*


<div class="alert alert-success" style="color: #333; background-color: #e6e6fa; border-color: #d8bfd8; border-width: 1px; border-radius: 3px; padding: 10px;">

**3.2.1**  Generate your test-set class predictions using your regularized model. Save those predictions to a `.csv` formatted file. Submit that `.csv` file [to the CS1090B Kaggle Competition](https://www.kaggle.com/competitions/cs-1090-b-hw-5-contest-2025/) for leaderboard scoring. 

**IMPORTANT:** For Kaggle to accept and score your submitted `.csv` file, it MUST contain 2 columns. The first column must be named `"id"` and needs to contain the test observation index numbers corresponding to each of your 54,000 predictions, the second column must be named `"label"` and needs to contain your class predictions (i.e. `cat` or `dog` etc.) for each corresponding test observation index location. A sample submission file is provided on [Kaggle](https://www.kaggle.com/competitions/cs-1090-b-hw-5-contest-2025/data?select=sample_submission.csv)
</div>

In [4]:
# your code here



<div class="alert alert-success" style="color: #333; background-color: #e6e6fa; border-color: #d8bfd8; border-width: 1px; border-radius: 3px; padding: 10px;">


**3.2.2**  **Specify your Kaggle name that you have used on the leaderboard**. We CANNOT give you credit without this.

</div>

**YOUR KAGGLE LEADERBOARD NAME:**

*Your answer here*



<!-- BEGIN QUESTION -->


<div class="alert alert-success" style="color: #333; background-color: #e6e6fa; border-color: #d8bfd8; border-width: 1px; border-radius: 3px; padding: 10px;">
<b>3.3 Wrap-up</b></div>

* In a few sentences, please describe the aspect(s) of the assignment you found most challenging. This could be conceptual and/or related to coding and implementation.

* How many hours did you spend working on this assignment? Store this as an int or float in `hours_spent_on_hw`. If you worked on the project in a group, report the *average* time spent per person.

*Your answer here*

In [ ]:
hours_spent_on_hw = ... # np.inf

In [ ]:
time_end = time.time()
print(f"It took {(time_end - time_start)/60:.2f} minutes for this notebook to run")

**This concludes HW5... the final CS 1090B HW! Thank you!**